In [ ]:
from kafka import KafkaConsumer
import psycopg2
import os
from dotenv import load_dotenv

In [ ]:
PG_HOST=os.getenv("PG_HOST")
PG_PORT=os.getenv("PG_PORT")
PG_USER=os.getenv("PG_USER")
PG_PASSWORD=os.getenv("PG_PASSWORD")
PG_DATABASE=os.getenv("PG_DATABASE")

In [ ]:
TOPIC_NAME = os.getenv("KAFKA_TOPIC1")
KAFKA_BOOTSTRAP_SERVER = ['localhost:9092']

In [ ]:
consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVER,
    auto_offset_reset='earliest'
)

In [ ]:
conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    user=PG_USER,
    password=PG_PASSWORD,
    database=PG_DATABASE
)
cur = conn.cursor()

In [ ]:
insert_query = """
INSERT INTO metrics (id, cpu, mem, disk)
VALUES (%s, %s, %s, %s)
"""

In [ ]:
for message in consumer:
    data = message.value

    value = message.value.decode('utf-8')
    numbers = tuple([int(item.split(":")[1].strip()) for item in value.split(",")])
 
    # Insert data into PostgreSQL table
    cur.execute(insert_query,numbers)
    conn.commit()

    print(f"Received message:value = {numbers}")


In [ ]:
# Close connections
consumer.close()
cur.close()
conn.close()